In [1]:
%run /Users/tyamgin/Projects/mlbootcamp/championship19/cv.ipynb
input_path = '/Users/tyamgin/Projects/mlbootcamp/championship17/data'
output_path = '/Users/tyamgin/Projects/mlbootcamp/championship17/res'

/usr/local/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)
Using TensorFlow backend.


In [2]:
train_data = parquet.read_table(input_path + '/collabTrain').to_pandas()

/usr/local/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [3]:
test_data = parquet.read_table(input_path + '/collabTest').to_pandas()

In [4]:
train_data.head(40)

,instanceId_userId,instanceId_objectType,instanceId_objectId,audit_pos,audit_clientType,audit_timestamp,audit_timePassed,audit_experiment,audit_resourceType,metadata_ownerId,metadata_ownerType,metadata_createdAt,metadata_authorId,metadata_applicationId,metadata_numCompanions,metadata_numPhotos,metadata_numPolls,metadata_numSymbols,metadata_numTokens,metadata_numVideos,metadata_platform,metadata_totalVideoLength,metadata_options,relationsMask,userOwnerCounters_USER_FEED_REMOVE,userOwnerCounters_USER_PROFILE_VIEW,userOwnerCounters_VOTE_POLL,userOwnerCounters_USER_SEND_MESSAGE,userOwnerCounters_USER_DELETE_MESSAGE,userOwnerCounters_USER_INTERNAL_LIKE,userOwnerCounters_USER_INTERNAL_UNLIKE,userOwnerCounters_USER_STATUS_COMMENT_CREATE,userOwnerCounters_PHOTO_COMMENT_CREATE,userOwnerCounters_MOVIE_COMMENT_CREATE,userOwnerCounters_USER_PHOTO_ALBUM_COMMENT_CREATE,userOwnerCounters_COMMENT_INTERNAL_LIKE,userOwnerCounters_USER_FORUM_MESSAGE_CREATE,userOwnerCounters_PHOTO_MARK_CREATE,userOwnerCounters_PHOTO_VIEW,userOwnerCounters_PHOTO_PIN_BATCH_CREATE,userOwnerCounters_PHOTO_PIN_UPDATE,userOwnerCounters_USER_PRESENT_SEND,userOwnerCounters_UNKNOWN,userOwnerCounters_CREATE_TOPIC,userOwnerCounters_CREATE_IMAGE,userOwnerCounters_CREATE_MOVIE,userOwnerCounters_CREATE_COMMENT,userOwnerCounters_CREATE_LIKE,userOwnerCounters_TEXT,userOwnerCounters_IMAGE,userOwnerCounters_VIDEO,ownerUserCounters_USER_FEED_REMOVE,ownerUserCounters_USER_PROFILE_VIEW,ownerUserCounters_VOTE_POLL,ownerUserCounters_USER_SEND_MESSAGE,ownerUserCounters_USER_DELETE_MESSAGE,ownerUserCounters_USER_INTERNAL_LIKE,ownerUserCounters_USER_INTERNAL_UNLIKE,ownerUserCounters_USER_STATUS_COMMENT_CREATE,ownerUserCounters_PHOTO_COMMENT_CREATE,ownerUserCounters_MOVIE_COMMENT_CREATE,ownerUserCounters_USER_PHOTO_ALBUM_COMMENT_CREATE,ownerUserCounters_COMMENT_INTERNAL_LIKE,ownerUserCounters_USER_FORUM_MESSAGE_CREATE,ownerUserCounters_PHOTO_MARK_CREATE,ownerUserCounters_PHOTO_VIEW,ownerUserCounters_PHOTO_PIN_BATCH_CREATE,ownerUserCounters_PHOTO_PIN_UPDATE,ownerUserCounters_USER_PRESENT_SEND,ownerUserCounters_UNKNOWN,ownerUserCounters_CREATE_TOPIC,ownerUserCounters_CREATE_IMAGE,ownerUserCounters_CREATE_MOVIE,ownerUserCounters_CREATE_COMMENT,ownerUserCounters_CREATE_LIKE,ownerUserCounters_TEXT,ownerUserCounters_IMAGE,ownerUserCounters_VIDEO,membership_status,membership_statusUpdateDate,membership_joinDate,membership_joinRequestDate,owner_create_date,owner_birth_date,owner_gender,owner_status,owner_ID_country,owner_ID_Location,owner_is_active,owner_is_deleted,owner_is_abused,owner_is_activated,owner_change_datime,owner_is_semiactivated,owner_region,user_create_date,user_birth_date,user_gender,user_status,user_ID_country,user_ID_Location,user_is_active,user_is_deleted,user_is_abused,user_is_activated,user_change_datime,user_is_semiactivated,user_region,feedback,objectId,auditweights_ageMs,auditweights_closed,auditweights_ctr_gender,auditweights_ctr_high,auditweights_ctr_negative,auditweights_dailyRecency,auditweights_feedOwner_RECOMMENDED_GROUP,auditweights_feedStats,auditweights_friendCommentFeeds,auditweights_friendCommenters,auditweights_friendLikes,auditweights_friendLikes_actors,auditweights_hasDetectedText,auditweights_hasText,auditweights_isPymk,auditweights_isRandom,auditweights_likersFeedStats_hyper,auditweights_likersSvd_prelaunch_hyper,auditweights_matrix,auditweights_notOriginalPhoto,auditweights_numDislikes,auditweights_numLikes,auditweights_numShows,auditweights_onlineVideo,auditweights_partAge,auditweights_partCtr,auditweights_partSvd,auditweights_processedVideo,auditweights_relationMasks,auditweights_source_LIVE_TOP,auditweights_source_MOVIE_TOP,auditweights_svd_prelaunch,auditweights_svd_spark,auditweights_userAge,auditweights_userOwner_CREATE_COMMENT,auditweights_userOwner_CREATE_IMAGE,auditweights_userOwner_CREATE_LIKE,auditweights_userOwner_IMAGE,auditweights_userOwner_MOVIE_COMMENT_CREATE,auditweights_userOwner_PHOTO_COMMENT_CREATE,auditweights_userOwner_PHOTO_MARK_CREATE

In [8]:
def proc_data_inplace(data):  
    data['instanceId_objectType'] = np.array(data.instanceId_objectType.replace({
        'Post': 0,
        'Video': 1,
        'Photo': 2,
    }), np.int8)
    data['audit_clientType'] = np.array(data.audit_clientType.replace({
        'WEB': 0,
        'API': 1,
        'MOB': 2,
    }), np.int8)
    data['metadata_ownerType'] = np.array(data.metadata_ownerType.replace({
        'GROUP_OPEN_OFFICIAL': 0,
        'GROUP_OPEN': 1,
    }), np.int8)
    data['metadata_platform'] = np.array(data.metadata_platform.replace({
        'OTHER': 0,
        'WEB': 1,
        'ANDROID': 2,
        'IOS': 3,
        'MOB': 4,
        'WINPHONE': 5,
    }), np.int8)

    drop_columns = [
        # nan or single value, == userOwnerCounters_USER_PROFILE_VIEW
       'userOwnerCounters_VOTE_POLL',
       'userOwnerCounters_USER_SEND_MESSAGE',
       'userOwnerCounters_USER_DELETE_MESSAGE',
       'userOwnerCounters_USER_INTERNAL_LIKE',
       'userOwnerCounters_USER_INTERNAL_UNLIKE',
       'userOwnerCounters_USER_STATUS_COMMENT_CREATE',
       'userOwnerCounters_PHOTO_COMMENT_CREATE',
       'userOwnerCounters_MOVIE_COMMENT_CREATE',
       'userOwnerCounters_USER_PHOTO_ALBUM_COMMENT_CREATE',
       'userOwnerCounters_COMMENT_INTERNAL_LIKE',
       'userOwnerCounters_USER_FORUM_MESSAGE_CREATE',
       'userOwnerCounters_PHOTO_MARK_CREATE',
       'userOwnerCounters_PHOTO_VIEW',
       'userOwnerCounters_PHOTO_PIN_BATCH_CREATE',
       'userOwnerCounters_PHOTO_PIN_UPDATE',
       'userOwnerCounters_USER_PRESENT_SEND',

       # single value
       'metadata_applicationId',
        
       # bad col in test
       'auditweights_closed',
    ]
    
    for c in data.columns:
        print(c)
        if data[c].isna().all():
            drop_columns.append(c)
    print('Drop columns: %s' % str(drop_columns))
    data.drop(drop_columns, 1, inplace=True)
        
    for c in ('HAS_CHEATED_TEXT', 'HAS_COMPANIONS', 'HAS_DETECTED_TEXT',
       'HAS_MUSIC', 'HAS_PHOTOS', 'HAS_PINS', 'HAS_POLLS', 'HAS_TEXT',
       'HAS_URLS', 'HAS_VIDEOS', 'IS_CENSORED', 'IS_DISGUSTING',
       'IS_EXTERNAL_SHARE', 'IS_GIF', 'IS_INTERNAL_GROUP_SHARE',
       'IS_INTERNAL_SHARE', 'IS_PART_OF_ALBUM', 'IS_PART_OF_TOPIC',
       'IS_PROMO'):
        data[c.lower()] = np.array(data.metadata_options.apply(lambda x: 1 if c in x else 0).values, np.int8)
    data.drop('metadata_options', 1, inplace=True)
    
    if 'feedback' in data.columns:
        for c in ('Liked', 'Clicked', 'Viewed', 'Disliked', 'ReShared', 'Ignored', 'Commented', 'Complaint', 'Unliked'):
            data[c.lower()] = np.array(data['feedback'].apply(lambda x: 1 if c in x else 0).values, np.int8)
        data.drop('feedback', 1, inplace=True)
      
    data['membership_status'] = np.array(data.membership_status.replace({
        'A': 1,
        'P': 2,
        'I': 3,
        'Y': 4,
        'M': 5,
        'B': 6,
        'R': 7,
        '!': 8,
    }).fillna(0), np.int8)
    
    for c, ctype in zip(data.columns, data.dtypes):
        if ctype == 'float64':
            data[c] = np.array(data[c], np.float32)
    

    

In [9]:
proc_data_inplace(train_data)

In [10]:
proc_data_inplace(test_data)

In [2]:
train_data = pd.read_pickle(output_path + '/train_data')
test_data = pd.read_pickle(output_path + '/test_data')

In [11]:
test_data.dtypes

instanceId_userId                                 int32
instanceId_objectType                              int8
instanceId_objectId                               int32
audit_pos                                         int64
audit_clientType                                   int8
audit_timestamp                                   int64
audit_timePassed                                  int64
audit_experiment                                 object
audit_resourceType                                int64
metadata_ownerId                                  int32
metadata_ownerType                                 int8
metadata_createdAt                                int64
metadata_authorId                                 int32
metadata_applicationId                            int64
metadata_numCompanions                            int32
metadata_numPhotos                                int32
metadata_numPolls                                 int32
metadata_numSymbols                             

In [25]:
train_data.metadata_ownerType.unique()

array(['GROUP_OPEN_OFFICIAL', 'GROUP_OPEN'], dtype=object)

In [27]:
train_data.metadata_platform.unique()

array(['OTHER', 'WEB', 'ANDROID', 'IOS', 'MOB', 'WINPHONE'], dtype=object)

In [69]:
train_data.membership_status.unique()

array(['A', None, 'P', 'I', '!', 'Y', 'M', 'B', 'R'], dtype=object)

In [45]:
np.unique(np.concatenate(train_data.metadata_options.values))

array(['HAS_CHEATED_TEXT', 'HAS_COMPANIONS', 'HAS_DETECTED_TEXT',
       'HAS_MUSIC', 'HAS_PHOTOS', 'HAS_PINS', 'HAS_POLLS', 'HAS_TEXT',
       'HAS_URLS', 'HAS_VIDEOS', 'IS_CENSORED', 'IS_DISGUSTING',
       'IS_EXTERNAL_SHARE', 'IS_GIF', 'IS_INTERNAL_GROUP_SHARE',
       'IS_INTERNAL_SHARE', 'IS_PART_OF_ALBUM', 'IS_PART_OF_TOPIC',
       'IS_PROMO'], dtype=object)

In [70]:
np.unique(np.concatenate(train_data.feedback.values))

array(['Clicked', 'Commented', 'Complaint', 'Disliked', 'Ignored',
       'Liked', 'ReShared', 'Unliked', 'Viewed'], dtype=object)

In [ ]:
for c in train.columns:
    test_uniq = test[c].unique() if c in test.columns else np.array([1,2])
    train_uniq = train[c].unique()
    if len(train_uniq) == 1 or len(test_uniq) == 1:
        print('all ' + c)
    elif (~np.isnan(train_uniq)).sum() == 1 or (~np.isnan(test_uniq)).sum() == 1:
        print('all2 ' + c)

In [5]:
train_data.to_pickle(output_path + '/train_data')
test_data.to_pickle(output_path + '/test_data')